In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from data import clean_and_split_nba_data as clean
from data import split_data as split
from data import resample_nba_data as re
from models import plot_validation_curve as vc
from models import eval_model as evm
from sklearn.ensemble import RandomForestClassifier
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv("../data/raw/train.csv")
df

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,10556,3799,80,24.3,7.8,3.0,6.4,45.7,0.1,0.3,...,2.9,72.1,2.2,2.0,3.8,3.2,1.1,0.2,1.6,1
1,5342,3800,75,21.8,10.5,4.2,7.9,55.1,-0.3,-1.0,...,3.6,67.8,3.6,3.7,6.6,0.7,0.5,0.6,1.4,1
2,5716,3801,85,19.1,4.5,1.9,4.5,42.8,0.4,1.2,...,0.6,75.7,0.6,1.8,2.4,0.8,0.4,0.2,0.6,1
3,13790,3802,63,19.1,8.2,3.5,6.7,52.5,0.3,0.8,...,1.5,66.9,0.8,2.0,3.0,1.8,0.4,0.1,1.9,1
4,5470,3803,63,17.8,3.7,1.7,3.4,50.8,0.5,1.4,...,0.5,54.0,2.4,2.7,4.9,0.4,0.4,0.6,0.7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,2996,11794,32,9.2,1.8,0.7,1.8,40.3,-0.1,-0.2,...,0.6,65.7,0.3,1.8,1.9,0.5,0.3,0.2,0.4,1
7996,11679,11795,54,6.0,1.8,0.7,1.4,48.7,0.1,0.1,...,0.4,70.1,1.0,1.1,2.0,0.1,0.0,0.3,0.3,1
7997,5537,11796,85,28.2,10.7,4.0,9.0,45.1,0.2,0.6,...,3.9,69.7,1.0,2.1,3.1,3.4,1.2,0.2,1.8,1
7998,1292,11797,39,7.7,2.5,1.0,2.3,40.1,-0.3,-0.5,...,0.7,74.3,0.4,0.6,0.9,0.2,0.3,0.3,0.5,1


In [4]:
df_upsampled, df_downsampled = re.resample_nba_data(df)

df_minority.shape (1331, 22)  df_majority.shape (6669, 22)
df_minority.shape 0    1331
Name: TARGET_5Yrs, dtype: int64  df_majority.shape 1    6669
Name: TARGET_5Yrs, dtype: int64
df_upsampled  0    6669
1    6669
Name: TARGET_5Yrs, dtype: int64
0    6669
1    6669
Name: TARGET_5Yrs, dtype: int64
df_downsampled  0    1331
1    1331
Name: TARGET_5Yrs, dtype: int64
0    1331
1    1331
Name: TARGET_5Yrs, dtype: int64


In [7]:
x_data, x_train, x_val, x_test, y_data , y_train, y_val,  y_test = clean.clean_and_split_nba_data(df_downsampled)

Index(['GP', 'MIN', 'PTS', 'FGM', 'FGA', '3P Made', '3PA', 'FTM', 'FTA',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'CALC3P%', 'CALCFG%',
       'CALCFT%'],
      dtype='object')


In [8]:
x_data.head()

,GP,MIN,PTS,FGM,FGA,3P Made,3PA,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,CALC3P%,CALCFG%,CALCFT%
217,95,24.5,13.3,5.0,12.1,0.7,2.1,2.4,2.9,0.4,1.3,1.7,3.3,1.1,0.0,1.7,33.333333,41.322314,82.758621
1003,57,20.0,5.1,1.8,4.7,0.9,2.3,0.8,0.9,0.1,1.1,1.2,4.3,0.7,0.1,1.3,39.130435,38.297872,88.888889
994,86,17.4,6.9,2.8,7.0,0.0,0.0,1.4,2.2,1.4,3.0,4.5,0.5,0.5,0.3,0.7,0.000000,40.000000,63.636364
759,63,9.4,4.2,1.5,3.5,0.4,1.0,1.0,1.7,0.3,0.5,0.9,2.3,1.0,0.0,0.8,40.000000,42.857143,58.823529
1881,38,23.1,10.2,4.1,7.8,0.6,1.8,1.8,2.5,1.7,2.9,4.8,1.2,0.5,0.4,1.1,33.333333,52.564103,72.000000


In [13]:
np.isna(x_data)

AttributeError: module 'numpy' has no attribute 'isna'

In [16]:
x_test.isna().sum()* 100 / len(df)

GP         0.0
MIN        0.0
PTS        0.0
FGM        0.0
FGA        0.0
3P Made    0.0
3PA        0.0
FTM        0.0
FTA        0.0
OREB       0.0
DREB       0.0
REB        0.0
AST        0.0
STL        0.0
BLK        0.0
TOV        0.0
CALC3P%    0.0
CALCFG%    0.0
CALCFT%    0.0
dtype: float64